In [4]:
date='0219'
filenumber='190'

In [5]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import regularizers
from keras.utils import plot_model

Using TensorFlow backend.


In [6]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.engine import training
from tensorflow.python.framework.ops import Tensor
import glob
import os
from typing import Tuple, List

In [7]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, Concatenate, Conv1D, Flatten, MaxPooling1D, Average
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA
from keras import initializers


In [8]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [9]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [10]:
x=x.astype('float32')
test_x=test_x.astype('float32')

In [11]:
# # Train_test_split
# In[7]:

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y, \
                                                    train_size=0.9, shuffle=True ,random_state=0)
                                                    

from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

from keras.utils import to_categorical

#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)


In [33]:
from sklearn.model_selection import KFold, StratifiedShuffleSplit

def get_stacking_data(x_tr, y_tr, x_te, n_folds=5):

    skfold=StratifiedShuffleSplit(n_splits=n_folds)
    
    tr_fold_predict=np.zeros((x_tr.shape[0],19))
    test_predict=np.zeros((x_te.shape[0],19))

    for cnt, (tr_index, val_index) in enumerate(skfold.split(x_tr,y_tr)):

        x_tr_ = x_tr[tr_index]
        y_tr_ = y_tr[tr_index]
        x_val_ = x_tr[val_index]
        
        inps1 = Input(shape = (20,))
        x1= Dense(128)(inps1)
        x1= Dense(128)(x1)        
        x1= Dense(128)(x1)
        x1= Dense(128)(x1)        
        outs= Dense(19,activation='softmax')(x1)
        
        models = Model(inputs=inps1, outputs=outs)
        models.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0001),metrics=['accuracy'])
        models.fit(x_tr, y_tr, batch_size=256, epochs=50, verbose=2)
    
        tr_fold_predict[val_index,:] = models.predict(x_val_) #.reshape(-1,5)
        test_predict += models.predict(x_te)

    test_predict_mean = (test_predict)/n_folds
    
    return tr_fold_predict, test_predict_mean

In [ ]:
x1_tr, x1_te = get_stacking_data(x_train, y_train, x_test, n_folds=8 )
x2_tr, x2_te = get_stacking_data(x_train, y_train, x_test, n_folds=8 )
x3_tr, x3_te = get_stacking_data(x_train, y_train, x_test, n_folds=8 )

Epoch 1/50
 - 3s - loss: 1.1049 - acc: 0.6908
Epoch 2/50
 - 2s - loss: 0.8197 - acc: 0.7603
Epoch 3/50
 - 2s - loss: 0.7868 - acc: 0.7681
Epoch 4/50
 - 2s - loss: 0.7752 - acc: 0.7700
Epoch 5/50
 - 2s - loss: 0.7668 - acc: 0.7732
Epoch 6/50
 - 2s - loss: 0.7619 - acc: 0.7736
Epoch 7/50
 - 2s - loss: 0.7568 - acc: 0.7756
Epoch 8/50
 - 2s - loss: 0.7534 - acc: 0.7761
Epoch 9/50
 - 2s - loss: 0.7504 - acc: 0.7769
Epoch 10/50
 - 2s - loss: 0.7484 - acc: 0.7772
Epoch 11/50
 - 2s - loss: 0.7468 - acc: 0.7778
Epoch 12/50
 - 2s - loss: 0.7455 - acc: 0.7784
Epoch 13/50
 - 2s - loss: 0.7433 - acc: 0.7786
Epoch 14/50
 - 2s - loss: 0.7423 - acc: 0.7791
Epoch 15/50
 - 2s - loss: 0.7404 - acc: 0.7796
Epoch 16/50
 - 2s - loss: 0.7401 - acc: 0.7791
Epoch 17/50
 - 2s - loss: 0.7390 - acc: 0.7797
Epoch 18/50
 - 2s - loss: 0.7384 - acc: 0.7794
Epoch 19/50
 - 2s - loss: 0.7380 - acc: 0.7792
Epoch 20/50
 - 2s - loss: 0.7380 - acc: 0.7796
Epoch 21/50
 - 2s - loss: 0.7380 - acc: 0.7796
Epoch 22/50
 - 2s - lo

In [ ]:
new_x_tr =np.concatenate((x1_tr, x2_tr, x3_tr), axis=1)
new_x_te =np.concatenate((x1_te, x2_te, x3_te), axis=1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf=RandomForestClassifier()
rf.fit(new_x_tr,y_train)
stack_pred=rf.predict(new_x_te)
print(accuracy_score(stack_pred,y_test))